# Coordinate descent for least squares

Author: Alexandre Gramfort, Mathurin Massias

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

## Generate simulated data

$y = Ax+noise$

In [ ]:
np.random.seed(0)
n_samples, n_features = 300, 100
A = np.random.randn(n_samples, n_features)
x = np.random.randn(n_features)
y = np.dot(A, x)
y += 0.3 * np.random.randn(n_samples)

## Option 1 : 

$\min_w \frac{1}{2} \|y - Ax\|^2$

solved with `linalg.solve`

In [ ]:
x_solve = linalg.solve(np.dot(A.T, A), np.dot(A.T, y))

In [ ]:
plt.scatter(x, x_solve);

## Option 2: gradient descent

In [ ]:
def gradient(x):
    return np.dot(A.T, np.dot(A, x) - y)


def objective(x):
    return 0.5 * linalg.norm(np.dot(A, x) - y) ** 2


L = linalg.norm(A, ord=2) ** 2

n_iter = 300

x_hat = np.zeros_like(x)
objs = list()
for k in range(n_iter):
    x_hat -= 1. / L * gradient(x_hat)
    objs.append(objective(x_hat))

eps = np.finfo('float').eps
plt.semilogy(np.maximum(np.array(objs) - objective(x_solve), eps))
plt.xlabel('iteration')
plt.ylabel('objective')

## Option 3: cyclic coordinate descent

In [ ]:
Li = np.sum(A ** 2, axis=0)

n_iter = 300 * n_features

x_hat = np.zeros_like(x)
objs = list()
for k in range(n_iter):
    i = k % n_features
    x_hat[i] = np.dot(A[:, i].T, y - np.dot(A, x_hat) + x_hat[i] * A[:, i])
    x_hat[i] /= np.dot(A[:, i], A[:, i])
    objs.append(objective(x_hat))

plt.semilogy(np.maximum(np.array(objs) - objective(x_solve), eps))
plt.xlabel('iteration')
plt.ylabel('objective')

### or written in a more generic manner:

In [ ]:
def gradient_i(x, i):
    return np.dot(A[:, i].T, np.dot(A, x) - y)

Li = np.sum(A * A, axis=0)

n_iter = 300 * n_features

x_hat = np.zeros_like(x)
objs = list()
for k in range(n_iter):
    i = k % n_features
    x_hat[i] -= 1. / Li[i] * gradient_i(x_hat, i)
    objs.append(objective(x_hat))

plt.semilogy(np.maximum(np.array(objs) - objective(x_solve), eps))
plt.xlabel('iteration')
plt.ylabel('objective')